In [1]:
import json

# Specify the GeoJSON file path
input_geojson = "data/contour_NYC_2ft.geojson"
output_geojson = "data/contour_NYC_2ft_extracted.geojson"

# Elevations to extract
target_elevations = ["-2 ft", "0 ft", "2 ft", "6 ft", "10 ft", "20 ft"]

# Load the GeoJSON file
with open(input_geojson, "r") as file:
    data = json.load(file)

# Extract features with matching elevations
filtered_features = [
    feature for feature in data["features"]
    if "ELEVATION" in feature["properties"] and feature["properties"]["ELEVATION"] in target_elevations
]

# Create a new GeoJSON object for the filtered features
filtered_geojson = {
    "type": "FeatureCollection",
    "features": filtered_features,
}

# Save the filtered GeoJSON to a new file
with open(output_geojson, "w") as file:
    json.dump(filtered_geojson, file, indent=2)

print(f"Extracted {len(filtered_features)} features with target elevations.")
print(f"Filtered GeoJSON saved to {output_geojson}")


Extracted 57832 features with target elevations.
Filtered GeoJSON saved to data/contour_NYC_2ft_extracted.geojson


In [1]:
import geopandas as gpd

# Path to your input and output GeoJSON files
input_geojson = "data/contour_NYC_2ft_extracted.geojson"
output_geojson = "data/contour_NYC_2ft_reprojected.geojson"

# Load the GeoJSON file into a GeoDataFrame
gdf = gpd.read_file(input_geojson)

# Check the current CRS (Coordinate Reference System)
print("Original CRS:", gdf.crs)

# Reproject to EPSG:4326 (WGS84) if not already in this projection
target_crs = "EPSG:4326"  # Use EPSG:3857 for Web Mercator if needed
gdf_reprojected = gdf.to_crs(target_crs)

# Save the reprojected GeoDataFrame to a new GeoJSON file
gdf_reprojected.to_file(output_geojson, driver="GeoJSON")

print(f"Reprojected GeoJSON saved to {output_geojson}")

Original CRS: EPSG:4326
Reprojected GeoJSON saved to data/contour_NYC_2ft_reprojected.geojson


In [2]:
import geopandas as gpd

# Load the GeoJSON file
gdf = gpd.read_file("data/contour_NYC_2ft_extracted.geojson")

# Check CRS and Geometry
print("CRS:", gdf.crs)
print("Geometry Issues:", gdf.is_valid.sum(), "/", len(gdf))


CRS: EPSG:4326
Geometry Issues: 57832 / 57832


In [3]:
import geopandas as gpd

# Load the GeoJSON file
gdf = gpd.read_file("data/contour_NYC_2ft_extracted.geojson")

# Check geometry validity
invalid_geometries = gdf[~gdf.is_valid]
print("Number of invalid geometries:", len(invalid_geometries))

# Save invalid geometries for inspection
invalid_geometries.to_file("data/invalid_geometries.geojson", driver="GeoJSON")


Number of invalid geometries: 0


In [5]:
from shapely.validation import make_valid

# Apply make_valid to all geometries
gdf["geometry"] = gdf.geometry.apply(make_valid)

# Check if geometries are now valid
print("Geometry Issues After Fix:", gdf.is_valid.sum(), "/", len(gdf))

# Save the fixed GeoJSON
gdf.to_file("data/fixed_contour.geojson", driver="GeoJSON")



Geometry Issues After Fix: 57832 / 57832


c:\Users\zheng\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\geopandas.py:523: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)
  has_z_arr = geometry[geometry.notna() & (~geometry.is_empty)].has_z


In [11]:
import geopandas as gpd

# Load GeoJSON
gdf = gpd.read_file("data/contour_NYC_2ft_extracted.geojson")

# Display bounds of geometries
print(gdf.geometry.bounds.head())

# Export raw data for manual inspection
gdf.to_file("data/raw_geometries.geojson", driver="GeoJSON")


          minx          miny         maxx          maxy
0  581658.5930  4.509639e+06  590366.2623  4.530358e+06
1  589911.9628  4.516752e+06  602055.4646  4.530046e+06
2  589933.7093  4.528819e+06  589935.0170  4.528821e+06
3  599070.8824  4.522560e+06  602556.8781  4.527434e+06
4  598830.9501  4.527131e+06  598833.2037  4.527133e+06


In [18]:
# Load GeoJSON file
gdf = gpd.read_file("data/contour_NYC_2ft_extracted.geojson")

# Set the correct CRS (EPSG:26918)
gdf = gdf.set_crs(epsg=26918, allow_override=True)

print("CRS successfully set to EPSG:26918.")

CRS successfully set to EPSG:26918.


In [19]:
# Re-project to WGS84 (EPSG:4326)
gdf_reprojected = gdf.to_crs(epsg=4326)

# Save the re-projected GeoJSON
gdf_reprojected.to_file("data/contour_NYC_2ft_extractedReprojected.geojson", driver="GeoJSON")

print("Re-projection to EPSG:4326 completed.")

Re-projection to EPSG:4326 completed.


In [20]:
# Load the GeoJSON
gdf = gpd.read_file("data/contour_NYC_2ft_extractedReprojected.geojson")

# Convert ELEVATION field to numeric
gdf["ELEVATION"] = gdf["ELEVATION"].str.replace(" ft", "").astype(float)

# Save the cleaned GeoJSON
gdf.to_file("data/cleaned_contour_NYC_2ft.geojson", driver="GeoJSON")

print("ELEVATION field cleaned and GeoJSON saved.")


ELEVATION field cleaned and GeoJSON saved.
